In [46]:
import glob
import pandas as pd
import numpy as np
import tensorflow as tf
import re
import nltk
import threading
import gensim
import logging
import pickle
import random
from gensim.models import FastText
from gensim.similarities.index import AnnoyIndexer
from gensim.models import Phrases
from gensim.models.phrases import Phraser
from gensim.models.word2vec import LineSentence

In [3]:
import glob
import os
import six
import threading
from gensim.models import KeyedVectors

from sortedcontainers import SortedDict
from collections import defaultdict, Counter
#from functools32 import lru_cache

In [4]:
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [5]:
path = '../data/newspapers/test/'
path2 = '../data/sentences'
allFiles = glob.glob(path + "/*.tsv")
tokenizer = nltk.data.load('tokenizers/punkt/dutch.pickle')

In [51]:
df = pd.concat((pd.read_csv(f, delimiter='\t') for f in allFiles))

In [52]:
df = df.dropna(subset =['ocr']) # remove lines with empty ocr field
df = df[~df['date'].str.contains('date')] #remove duplicate header rows
    # remove files that contain error msg 
excludes = ['objecttype', 'file directory not found']
df = df[~df['ocr'].astype(str).str.contains('|'.join(excludes))] #remove 

In [57]:
df

,date,page,size,min_x,min_y,max_x,max_y,w,h,image_url,ocr_url,ocr,year
0,1956-11-29,1,3719603,53,712,700,6460,647,5748,http://imageviewer.kb.nl/ImagingService/imagin...,http://resolver.kb.nl/resolve?urn=ABCDDD:01082...,"Convent en Bezit tVeHv Particuliere eigendom, ...",1956
1,1956-11-29,1,1031278,706,3096,1393,4641,687,1545,http://imageviewer.kb.nl/ImagingService/imagin...,http://resolver.kb.nl/resolve?urn=ABCDDD:01082...,"Werkgevers en werknemers: elk 1,3 % opvangen I...",1956
2,1956-11-29,1,1328731,81,3458,1389,4797,1308,1339,http://imageviewer.kb.nl/ImagingService/imagin...,http://resolver.kb.nl/resolve?urn=ABCDDD:01082...,Geen kostenstijging Doordat de vereveningsheff...,1956
3,1956-11-29,1,952556,2111,1564,2793,3018,682,1454,http://imageviewer.kb.nl/ImagingService/imagin...,http://resolver.kb.nl/resolve?urn=ABCDDD:01082...,M. Ruppert: „Verantwoord en verheugend” In een...,1956
4,1956-11-29,1,1810952,1407,3102,2796,4847,1389,1745,http://imageviewer.kb.nl/ImagingService/imagin...,http://resolver.kb.nl/resolve?urn=ABCDDD:01082...,Voorzitter prof. De Vries: S.E.R.-ADVIES VRUCH...,1956
5,1956-11-29,1,1860196,705,4957,2090,6495,1385,1538,http://imageviewer.kb.nl/ImagingService/imagin...,http://resolver.kb.nl/resolve?urn=ABCDDD:01082...,Na aftreden als kamerlid: Dr. J. Schouten lid ...,1956
6,1956-11-29,1,649936,2114,5396,2793,6458,679,1062,http://imageviewer.kb.nl/ImagingService/imagin...,http://resolver.kb.nl/resolve?urn=ABCDDD:01082...,CHINEES-INDIASE BEMIDDELING IN HONGARIJE? Een ...,1956
7,1956-11-29,1,4026936,709,748,3479,3062,2770,2314,http://imageviewer.kb.nl/ImagingService/imagin...,http://resolver.kb.nl/resolve?urn=ABCDDD:01082...,EENSTEMMIG S.E.R.-ADVIES AAN DE REGERING: Best...,1956
8,1956-11-29,1,420006,2109,4235,2795,4877,686,642,http://imageviewer.kb.nl/ImagingService/imagin...,http://resolver.kb.nl/resolve?urn=ABCDDD:01082...,"Ontspanning Prof. dr. F. L. Polak, die aan de ...",1956
9,1956-11-29,1,231371,2109,4923,2794,5311,685,388,http://imageviewer.kb.nl/ImagingService/imagin...,http://resolver.kb.nl/resolve?urn=ABCDDD:01082...,Landbouw Voor eventuele maatregelen ten dienst...,1956


In [53]:
df['date'] = pd.to_datetime(df['date'])

In [54]:
df['year'] = df['date'].dt.year

In [56]:
df['sentences'] = df['ocr'].apply(article_to_sentences)

KeyboardInterrupt: 

In [35]:
def cleaning(text):
    text = re.sub("[^a-zA-Z]"," ", text)
    words = text.lower().split()
    return(words)

def article_to_sentences(text):
    #raw_sentences = tokenizer.tokenize(text.strip())
    raw_sentences = nltk.sent_tokenize(text)
    #raw_sentences = text.split('.')
    sentences = []
    for raw_sentence in raw_sentences:
        if len(raw_sentence) > 0:
            sentences.append(cleaning(raw_sentence))
    return sentences

In [ ]:
def getSentencesForYear(year):
    corpus = []
    for file_ in allFiles:
        filename = os.path.basename(file_)
        print(filename)
        if filename.startswith(str(year)):
            df = pd.read_csv(file_, sep='\t', encoding='utf-8', header=None)
            df.columns = ['date', 'page', 'size', 'min_x', 'min_y', 'max_x', 'max_y', 'w', 'h', 'image_url', 'ocr_url', 'ocr']
            df = df[df['ocr'].notnull()] #remove empty rows
            sentences = df['ocr'].apply(article_to_sentences)
            sentences.to_csv(path2 + '/'+ 'sentences' + filename, index=False, sep='\t', encoding='utf-8')
            sentences = [item for sublist in sentences for item in sublist]
            bigram_transformer = gensim.models.Phrases(sentences)
            bigram = gensim.models.phrases.Phraser(bigram_transformer)
            corpus = list(bigram[sentences])
    return corpus
        

def getSentencesInRange(startY, endY):
    return [s for year in range(startY, endY) for s in getSentencesForYear(year)]

In [ ]:
for file_ in allFiles:
        filename = os.path.basename(file_)
        print(filename)
        if filename.startswith(str(1990)):
            df = pd.read_csv(file_, sep='\t', encoding='utf-8')

In [ ]:
def train_models():
    yearsInModel = 49    
    stepYears = 49
    modelFolder = '../models/tempModel_Trouw/'
    data = []
    y0 = 1946
    yN = 1995

    for year in range(y0, yN-yearsInModel+1, stepYears):
        startY = year
        endY = year + yearsInModel
        modelName = modelFolder + '/%d_%d.w2v'%(year,year+yearsInModel)
        print('Building Model: ', modelName)
        
        corpus = getSentencesInRange(startY, endY)

        #model = gensim.models.Word2Vec(min_count=10, size = 200, iter=50, alpha=0.025, window = 10, workers = 40, sg=1)
        #model.build_vocab(corpus)
        #model.train(corpus , total_examples=model.corpus_count, epochs=model.iter)
        #print('....saving')
        #model.init_sims(replace=True)
        #model.wv.save_word2vec_format(modelName, binary=True)

In [ ]:
train_models()

In [ ]:
model.most_similar('verenigde_staten')

In [ ]:
#model.save('/tmp/model_1975')
model_1975 = gensim.models.Word2Vec.load('/tmp/model_1975')

In [ ]:
model.most_similar('kernoorlog')

In [ ]:
#model = gensim.models.Word2Vec(min_count=10, size = 200, iter=10, alpha=0.025, window = 10, workers = 7)
model.build_vocab(sentences_1985, update=True)
model.train(sentences_1985, total_examples=model.corpus_count, epochs=model.iter)
#model.init_sims(replace=True) 

In [ ]:
#model_1975.most_similar('man')
model.most_similar('man')

In [ ]:
def _getRelatedTerms(model, seedTerms, maxRelatedTerms):
    queries = []
    threads = []

    for term in seedTerms:
        t = threading.Thread(target=_getRelatedTermsThread,
                             args=(model, term, maxRelatedTerms, queries))
        threads.append(t)
        t.start()
    for t in threads:
        t.join()
    queries.sort()
    return queries

def _getRelatedTermsThread(model, term, maxRelatedTerms, queries,
                           ):
    try:
        newTerms = model.most_similar(term, topn=maxRelatedTerms)

        # list.append is thread safe, so we should be ok
        queries.append((term, newTerms))
    except KeyError:
        queries.append((term, []))
        pass

def _pruned(pairs, words):
    '''Returns a list of (word, weight) tuples which is the same as the given
    list (pairs), except containing only words in the set (words)'''
    return [(word, weight) for word, weight in pairs if word in words]


def _getCommonTerms(terms, N):
    '''Return a the top N terms of the given list. Terms are given as a
    dictionary of { term: weight } and the top terms are the terms with the
    highest weights.'''
    termCounter = Counter(terms)
    topTerms = termCounter.most_common(N)
    return topTerms


    


In [ ]:
def _trackCore(model, seedTerms, maxTerms=250, maxRelatedTerms=10,
                   minSim=0.4, wordBoost=1.0, reward=lambda x: 1.0):
        '''Given a list of seed terms, queries the given model to produce a
        list of terms. A dictionary of links is also returned as a dictionary:
        { seed: [(word,weight),...]}'''
        dRelatedTerms = defaultdict(float)
        links = defaultdict(list)

        relatedTermQueries = _getRelatedTerms(
            model, seedTerms, maxRelatedTerms)

        # Get the first tier related terms
        for term, newTerms in relatedTermQueries:
            # The terms are always related to themselves
            dRelatedTerms[term] = wordBoost
            links[term].append((term, 0.0))

            for newTerm, tSim in newTerms:
                if tSim < minSim:
                    break
                dRelatedTerms[newTerm] += reward(tSim)
                links[term].append((newTerm, tSim))

        # Select the top N terms with biggest weights (where N=maxTerms)
        topTerms = _getCommonTerms(dRelatedTerms, maxTerms)

        selectedTerms = set(word for word, weight in topTerms)
        links = {seed: _pruned(pairs, selectedTerms)
                 for seed, pairs in links.items()}
        return topTerms, links

In [ ]:
seedTerms = ['president']

In [ ]:
_trackCore(model_1985, seedTerms)

In [ ]:
women_vector = 0
for term in topTerms:
    women_vector += model_vk[term[0]]

In [ ]:
model_vk.similar_by_vector(women_vector, topn=20, restrict_vocab=None)

In [ ]:
annoy_index = AnnoyIndexer(model, 1000)

In [ ]:
model_nrc.most_similar(['politiek'], topn=15)

In [ ]:
model_vk.most_similar(['politiek'], topn=15, indexer=annoy_index)


In [ ]:
def get_vec(model, word):
    if word in model.vocab:
        vec = model[word]
    else:
        vec = None
    return vec
    

In [ ]:
#cosine similarity two words
model.wv.similarity('amerika', 'oorlog')

In [ ]:
model.wv.n_similarity(['amerika','president'],['mens','abortus'])

In [ ]:
model.wv.doesnt_match(['amerika','frisdrank','president'])

In [ ]:
#model.most_similar(positive=['vrouw'], negative=['man'], topn=50)
model.most_similar(positive=['man'], negative=['vrouw'], topn=50)

In [ ]:
model.most_similar(positive=['vrouw', 'vrouwen','man','mannen'], topn=50)

In [ ]:
her_tokens = [token for token,weight in model.wv.most_similar(positive=['vrouw'], negative=['man'], topn=50)]

In [ ]:
her_tokens

In [ ]:
# Get the vector for each sampled word
vectors = [model[word] for word in her_tokens]

In [ ]:
from sklearn.metrics import pairwise
dist_matrix = pairwise.pairwise_distances(vectors, metric='cosine')

In [ ]:
from sklearn.manifold import MDS
mds = MDS(n_components = 2, dissimilarity='precomputed')
embeddings = mds.fit_transform(dist_matrix)

In [ ]:
import matplotlib.pyplot as plt
_, ax = plt.subplots(figsize=(10,10))
ax.scatter(embeddings[:,0], embeddings[:,1], alpha=0)
for i in range(len(vectors)):
    ax.annotate(her_tokens[i], ((embeddings[i,0], embeddings[i,1])))

In [ ]:

# Create functions that reject vectors, find nearest words

def vector_rejector(model, word1, word2):
    vector1 = model[word1]
    vector2 = model[word2]
    # Rejects vector_2 from vector_1
    # Note that vectors are passed in, not words; also mean of
    # several vectors could be passed as a single argument
    projection = np.dot(vector1,vector2) * vector2
    return vector1 - projection

In [ ]:
she_vector = vector_rejector(model, 'vrouw', 'man')

In [ ]:
model.similar_by_vector(she_vector, topn=20, restrict_vocab=None)